In [1]:
import os 
import cv2
import pandas as pd
import numpy as np 
from tqdm import tqdm
import Functions as func
import var
import pickle 
import random

Using plaidml.keras.backend backend.


The general flow of our algorithm will be:

1. **Step #1:** Input an image
2. **Step #2:** Construct an image pyramid
3. **Step #3:** For each scale of the image pyramid, run a sliding window
4. **Step #3a:** For each stop of the sliding window, extract the ROI
5. **Step #3b:** Take the ROI and pass it through our CNN originally trained for image classification
6. **Step #3c:** Examine the probability of the top class label of the CNN, and if meets a minimum confidence, record (1) the class label and (2) the location of the sliding window
7. **Step #4:** Apply class-wise non-maxima suppression to the bounding boxes
8. **Step #5:** Return results to calling function

In [2]:
def get_iou(box1, box2):
    # determine the (x, y)-coordinates of the intersection rectangle
    xa = max(box1[0], box2[0])
    ya = max(box1[1], box2[1])
    xb = min(box1[2], box2[2])
    yb = min(box1[3], box2[3])
    # compute the area of intersection rectangle
    interArea = max(0, xb - xa + 1) * max(0, yb - ya + 1)
    
    # compute the area of both the prediction and ground-truth
    # rectangles
    boxAArea = (box1[2] - box1[0] + 1) * (box1[3] - box1[1] + 1)
    boxBArea = (box2[2] - box2[0] + 1) * (box2[3] - box2[1] + 1)
    
    iou = interArea / float(boxAArea + boxBArea - interArea)
    # return the intersection over union value
    return iou

In [3]:
folder ='Separated'
df = pd.read_csv(f'../{folder}/PATHS.csv').drop('Unnamed: 0', axis =1)

zipped_paths = df[['filename']]

max_prop = 2000
max_pos = 2
max_neg = 3
input_dim = var.norm_dimension
total_pos = 0 
total_neg = 0
error_array = [] 
total_max_neg = 5000
pbar = tqdm(range(len(df)))
for idx in pbar: 
    row = df.iloc[idx]
    img_path = row.filename
    pbar.set_description(f'{img_path}')
    try:
        image = cv2.imread(f'../Separated/Pistol/{img_path}')
        ss = cv2.ximgproc.segmentation.createSelectiveSearchSegmentation()
        ss.setBaseImage(image)
        ss.switchToSelectiveSearchFast()
        rects = ss.process() 
        prop_rect = [] 

        xMin = row.xmin
        xMax = row.xmax
        yMin = row.ymin
        yMax = row.ymax
        gt_box = ((xMin, yMin, xMax, yMax))
        gt_startx, gt_starty, gt_endx, gt_endy = gt_box
        for (x,y,w,h) in rects: 
            prop_rect.append((x,y,x+w, y+h))
        positive = 0 
        negative = 0 
        for rect in prop_rect[:1000]: 
            startx, starty, endx, endy = rect
            iou = get_iou(gt_box, rect)
            if iou > .7 and positive < max_pos: 
                roi = image[starty:endy, startx:endx]
                filename = f'{total_pos}.png'
                positive += 1
                total_pos += 1 
                roi = cv2.resize(roi, input_dim, interpolation = cv2.INTER_CUBIC)
                cv2.imwrite(f'../Separated/FinalImages/Pistol/{filename}', roi)
                continue

            fullOverlap = startx >= gt_startx
            fullOverlap = fullOverlap and starty >= gt_starty
            fullOverlap = fullOverlap and endx <= gt_endx
            fullOverlap = fullOverlap and endy <= gt_endy

            if not fullOverlap and iou == 0 and negative <= max_neg: 
                roi = image[starty:endy, startx: endx]
                filename = f'{total_neg}.png'
                negative += 1
                total_neg += 1
                roi = cv2.resize(roi, input_dim, interpolation = cv2.INTER_CUBIC)
                cv2.imwrite(f'../Separated/FinalImages/NoWeapon/{filename}', roi)
                continue 
    except: 
        try: 
            image = cv2.imread(f'../Separated/AR/{img_path}')
            ss = cv2.ximgproc.segmentation.createSelectiveSearchSegmentation()
            ss.setBaseImage(image)
            ss.switchToSelectiveSearchFast()
            rects = ss.process() 
            prop_rect = [] 

            xMin = row.xmin
            xMax = row.xmax
            yMin = row.ymin
            yMax = row.ymax
            gt_box = ((xMin, yMin, xMax, yMax))
            gt_startx, gt_starty, gt_endx, gt_endy = gt_box
            for (x,y,w,h) in rects: 
                prop_rect.append((x,y,x+w, y+h))
            positive = 0 
            negative = 0 
            for rect in prop_rect[:1000]: 
                startx, starty, endx, endy = rect
                iou = get_iou(gt_box, rect)
                if iou > .7 and positive < max_pos: 
                    roi = image[starty:endy, startx:endx]
                    filename = f'{total_pos}.png'
                    positive += 1
                    total_pos += 1 
                    roi = cv2.resize(roi, input_dim, interpolation = cv2.INTER_CUBIC)
                    cv2.imwrite(f'../Separated/FinalImages/Rifle/{filename}', roi)
                    continue

                fullOverlap = startx >= gt_startx
                fullOverlap = fullOverlap and starty >= gt_starty
                fullOverlap = fullOverlap and endx <= gt_endx
                fullOverlap = fullOverlap and endy <= gt_endy

                if not fullOverlap and iou == 0 and negative <= max_neg: 
                    roi = image[starty:endy, startx: endx]
                    filename = f'{total_neg}.png'
                    negative += 1
                    total_neg += 1
                    roi = cv2.resize(roi, input_dim, interpolation = cv2.INTER_CUBIC)
                    cv2.imwrite(f'../Separated/FinalImages/NoWeapon/{filename}', roi)
                    continue 
        except: 
            error_array.append((img_path, image))
            continue
        
pickle.dump(error_array, open('../Pickles/error_array1.p', 'wb'))        
    
    



Pistol_996_72.jpg: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 6202/6202 [1:43:10<00:00,  1.00it/s]


In [ ]:
print(error_array)

In [ ]:
assert False

In [15]:

##SLIDING WINDOW VIZ 
import time
def sliding_window(image, step_size, windowsize): 
    for y in range(0, image.shape[0], step_size):
        for x in range(0, image.shape[1], step_size):
#             cv2.rectangle(image, (x, y), (x + windowsize[0], y + windowsize[1]), (255,0,0), 2)
#     cv2.imshow('test', image)
#     cv2.waitKey(0)
    
            yield (x,y,image[y:y + windowsize[1], x:x + windowsize[0]])

    
    
    
df = pd.read_csv('../Separated/PATHS.csv')
img = cv2.imread('Pistol_1097_18.jpg')
df = df[df.filename == 'Pistol_1097_18.jpg']
origstartx = df.xmin.values[0]
origstarty = df.ymin.values[0] 
origendx = df.xmax.values[0] 
origendy  = df.ymax.values[0]

cv2.rectangle(img, (origstartx, origstarty), (origendx, origendy), (0,0,255), 2) 
# sliding_window(img, step_size = 30, windowsize = (30,30))

# windowsize = (80,80)
# for x, y, window in sliding_window(img, step_size = 30, windowsize = windowsize): 
#     clone = img.copy()
#     cv2.rectangle(clone, (x,y), (x+ windowsize[0], y + windowsize[1]), (255, 0, 0), 2)
#     cv2.imshow('Window', clone)
#     cv2.waitKey(0)
#     time.sleep(.025)
# cv2.destroyAllWindows()   


In [3]:
def auto_canny(image, sigma=0.33):
    # compute the median of the single channel pixel intensities
    v = np.median(image)
    # apply automatic Canny edge detection using the computed median
    lower = int(max(0, (1.0 - sigma) * v))
    upper = int(min(255, (1.0 + sigma) * v))
    edged = cv2.Canny(image, lower, upper)
    # return the edged image
    return edged
# roi = cv2.resize(roi, input_dim, interpolation = cv2.INTER_CUBIC)

img = cv2.imread('../TestImages/AR.jpg')
print(img.shape)
#gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
blurred = cv2.GaussianBlur(img, (3,3), 0)
wide = cv2.Canny(blurred, 10,200)
tight = cv2.Canny(blurred, 225, 250)
auto = auto_canny(blurred)
# wide = cv2.resize(wide, (var.vgg_dimension[0], var.vgg_dimension[1]), interpolation = cv2.INTER_CUBIC)
print(wide.shape)
cv2.imshow('Original', img)
cv2.imshow('Edges', np.hstack([wide, tight, auto]))
cv2.waitKey(0)

(317, 338, 3)
(317, 338)


-1

In [ ]:
(317, 338)


In [ ]:
# folder ='Separated'
# df = pd.read_csv(f'../{folder}/PATHS.csv').drop('Unnamed: 0', axis =1)

# zipped_paths = df[['filename']]

# max_prop = 2000
# max_pos = 5
# max_neg = 5
# input_dim = var.norm_dimension
# total_pos = 0 
# total_neg = 0
# error_array = [] 
# for idx in tqdm(range(len(df))): 
#     row = df.iloc[idx]
#     img_path = row.filename
#     image = cv2.imread(f'../{folder}/{img_path}')
#     try:
#         ss = cv2.ximgproc.segmentation.createSelectiveSearchSegmentation()
#         ss.setBaseImage(image)
#         ss.switchToSelectiveSearchFast()
#         rects = ss.process() 
#         prop_rect = [] 

#         xMin = row.xmin
#         xMax = row.xmax
#         yMin = row.ymin
#         yMax = row.ymax
#         gt_box = ((xMin, yMin, xMax, yMax))
#         gt_startx, gt_starty, gt_endx, gt_endy = gt_box
#         for (x,y,w,h) in rects: 
#             prop_rect.append((x,y,x+w, y+h))
#         positive = 0 
#         negative = 0 
#         for rect in prop_rect[:max_prop]: 
#             startx, starty, endx, endy = rect
#             iou = get_iou(gt_box, rect)
#             if iou > .7 and positive < max_pos: 
#                 roi = image[starty:endy, startx:endx]
#                 filename = f'{total_pos}.png'
#                 positive += 1
#                 total_pos += 1 
#                 roi = cv2.resize(roi, input_dim, interpolation = cv2.INTER_CUBIC)
# #                 cv2.imwrite(f'../FinalImages/Positive/{filename}', roi)
#                 continue

#             fullOverlap = startx >= gt_startx
#             fullOverlap = fullOverlap and starty >= gt_starty
#             fullOverlap = fullOverlap and endx <= gt_endx
#             fullOverlap = fullOverlap and endy <= gt_endy

#             if not fullOverlap and iou < .05 and negative <= max_neg: 
#                 roi = image[starty:endy, startx: endx]
#                 filename = f'{total_neg}.png'
#                 negative += 1
#                 total_neg += 1
#                 roi = cv2.resize(roi, input_dim, interpolation = cv2.INTER_CUBIC)
# #                 cv2.imwrite(f'../FinalImages/Negative/{filename}', roi)
#                 continue 
#     except: 
#         error_array.append((img_path, image))
#         raise
        
        
    
    

